In [1]:
#tweet cleaning
#https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
#https://towardsdatascience.com/basic-data-cleaning-engineering-session-twitter-sentiment-data-95e5bd2869ec

#Topic Modeilling
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

#Cleaning
#Puncuation
#lower case
#stopwords
#abbreviations

# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
#import spacy
#from scispacy.abbreviation import AbbreviationDetector

#sentiment analysis
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\megha\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
diabetes_all = pd.read_csv('diabetes_all_lda.csv')

In [4]:
# Print head
diabetes_all.head(10)

,Post ID,Author,Publish Date,Text,text_lemmatized,Permalink,Title,month_year,Dominant
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,"['happy', 'new', 'year', 'want', 'diagnose', 'type', 'dm', 'thursday', 'crazy', 'fatigue', 'thir...",/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,2018-01,Topic 4
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...","['lb', 'male', 'type', 'look', 'keto', 'way', 'lose', 'weight', 'hopefully', 'low', 'medication'...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",2018-01,Topic 4
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,"['ac', 'checked', 'today', 'threefour', 'month', 'take', 'metformin', 'ill', 'test', 'month', 'e...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,2018-01,Topic 4
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...","['old', 'post', 'congrats', 'motivate', 'note', 'desk', 'cycle', 'get', 'sure', 'reasonable', 't...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,2018-03,Topic 1
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,"['stick', 'fitbit', 'ankle', 'count', 'step', 'end', 'entire', 'time', 'desk', 'day', 'hour', 'u...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,2018-05,Topic 1
5,7nnogn,jdong4321,2018-01-02 9:02,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...","['mom', 'check', 'lot', 'symptom', 'type', 'month', 'lazyin', 'denial', 'know', 'actual', 'idiot...",/r/diabetes/comments/7nnogn/i_think_i_have_diabetes_but_im_overseas/,"I think I have diabetes but I'm overseas travelling for another 45 days, how urgent is it?",2018-01,Topic 4
6,7npao8,douglas4321,2018-01-02 12:51,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...","['type', 'insulin', 'doctor', 'bed', 'heard', 'people', 'best', 'directly', 'large', 'meal', 'wo...",/r/diabetes/comments/7npao8/when_is_the_best_time_to_take_my_10mg_jardiance/,when is the best time to take my 10mg jardiance Pill?,2018-01,Topic 12
7,7ntfa9,SeanP93,2018-01-03 0:43,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...","['yearsold', 'couple', 'pound', 'shy', 'pound', 'stranger', 'saw', 'assume', 'healthy', 'young',...",/r/diabetes/comments/7ntfa9/new_young_type_ii_here_looking_for_help_guidance/,"New, young type II here.. looking for help, guidance, advice, and comfort.",2018-01,Topic 14
8,7o0j4z,floyd2168,2018-01-03 20:26,I'm a recently diagnosed Type 2 Diabetic (Nov 2017). My prescription coverage dropped Jardiance ...,"['recently', 'diagnose', 'type', 'nov', 'prescription', 'coverage', 'drop', 'jardiance', 'formul...",/r/diabetes/comments/7o0j4z/prescription_coverage_forcing_a_change_to_farxiga/,Prescription coverage forcing a change to Farxiga. Wondering if anyone has experience with it.,2018-01,Topic 4
9,7o5gve,dianthe,2018-01-04 11:41,"Hi everyone, I need some advice. I have a friend who is having a major surgery on her knee in a ...","['need', 'advice', 'friend', 'major', 'surgery', 'knee', 'couple', 'week', 'want', 'bunch', 'foo...",/r/diabetes/comments/7o5gve/ideas_for_diabetic_friendly_recipes_which_can_be/,Ideas for diabetic friendly recipes which can be frozen?,2018-01,Topic 14


In [5]:
list(diabetes_all.columns)

['Post ID',
 'Author',
 'Publish Date',
 'Text',
 'text_lemmatized',
 'Permalink',
 'Title',
 'month_year',
 'Dominant']

In [6]:
#drop unnecessary columns
diabetes_all = diabetes_all.drop(columns = ['text_lemmatized'])
diabetes_all

,Post ID,Author,Publish Date,Text,Permalink,Title,month_year,Dominant
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,2018-01,Topic 4
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",2018-01,Topic 4
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,2018-01,Topic 4
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,2018-03,Topic 1
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,2018-05,Topic 1
...,...,...,...,...,...,...,...,...
85260,goxee5q,satxlonghorn1,2021-02-26 18:26,"I didn't have any side effects, just fyi. Some people don't.",/r/type2diabetes/comments/lsrxyw/new_here/goxee5q/,NaN,2021-02,Topic 12
85261,goxfko5,AccidentalDragon,2021-02-26 18:36,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,/r/type2diabetes/comments/lsrxyw/new_here/goxfko5/,NaN,2021-02,Topic 5
85262,lthxwe,Wackydetective,2021-02-26 22:17,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,/r/type2diabetes/comments/lthxwe/54_a1c/,5.4 A1C!!!,2021-02,Topic 8
85263,lu13pm,Nervous_Finger1486,2021-02-27 16:27,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,/r/type2diabetes/comments/lu13pm/still_new_to_this/,Still new to this.....,2021-02,Topic 8


In [7]:
#remove any blank texts
diabetes_all = diabetes_all[diabetes_all['Text'].notna()]
diabetes_all

,Post ID,Author,Publish Date,Text,Permalink,Title,month_year,Dominant
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,2018-01,Topic 4
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",2018-01,Topic 4
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,2018-01,Topic 4
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,2018-03,Topic 1
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,2018-05,Topic 1
...,...,...,...,...,...,...,...,...
85260,goxee5q,satxlonghorn1,2021-02-26 18:26,"I didn't have any side effects, just fyi. Some people don't.",/r/type2diabetes/comments/lsrxyw/new_here/goxee5q/,NaN,2021-02,Topic 12
85261,goxfko5,AccidentalDragon,2021-02-26 18:36,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,/r/type2diabetes/comments/lsrxyw/new_here/goxfko5/,NaN,2021-02,Topic 5
85262,lthxwe,Wackydetective,2021-02-26 22:17,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,/r/type2diabetes/comments/lthxwe/54_a1c/,5.4 A1C!!!,2021-02,Topic 8
85263,lu13pm,Nervous_Finger1486,2021-02-27 16:27,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,/r/type2diabetes/comments/lu13pm/still_new_to_this/,Still new to this.....,2021-02,Topic 8


In [9]:
#remove posts with less than 5 words
word_count = diabetes_all['Text'].str.split().str.len()
~(word_count < 5)

0        True
1        True
2        True
3        True
4        True
         ... 
85260    True
85261    True
85262    True
85263    True
85264    True
Name: Text, Length: 85265, dtype: bool

In [10]:
diabetes_all = diabetes_all [~(word_count < 5)]
diabetes_all

,Post ID,Author,Publish Date,Text,Permalink,Title,month_year,Dominant
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,2018-01,Topic 4
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",2018-01,Topic 4
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,2018-01,Topic 4
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,2018-03,Topic 1
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,2018-05,Topic 1
...,...,...,...,...,...,...,...,...
85260,goxee5q,satxlonghorn1,2021-02-26 18:26,"I didn't have any side effects, just fyi. Some people don't.",/r/type2diabetes/comments/lsrxyw/new_here/goxee5q/,NaN,2021-02,Topic 12
85261,goxfko5,AccidentalDragon,2021-02-26 18:36,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,/r/type2diabetes/comments/lsrxyw/new_here/goxfko5/,NaN,2021-02,Topic 5
85262,lthxwe,Wackydetective,2021-02-26 22:17,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,/r/type2diabetes/comments/lthxwe/54_a1c/,5.4 A1C!!!,2021-02,Topic 8
85263,lu13pm,Nervous_Finger1486,2021-02-27 16:27,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,/r/type2diabetes/comments/lu13pm/still_new_to_this/,Still new to this.....,2021-02,Topic 8


In [11]:
types = diabetes_all.dtypes
types

Post ID         object
Author          object
Publish Date    object
Text            object
Permalink       object
Title           object
month_year      object
Dominant        object
dtype: object

In [13]:
#drop month_year
diabetes_all = diabetes_all.drop(['month_year'], axis = 1)

In [14]:
#convert date to date time object
diabetes_all['month_year'] = pd.to_datetime(diabetes_all['Publish Date']).dt.to_period('M')
diabetes_all

,Post ID,Author,Publish Date,Text,Permalink,Title,Dominant,month_year
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,Topic 4,2018-01
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",Topic 4,2018-01
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,Topic 4,2018-01
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,Topic 1,2018-03
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,Topic 1,2018-05
...,...,...,...,...,...,...,...,...
85260,goxee5q,satxlonghorn1,2021-02-26 18:26,"I didn't have any side effects, just fyi. Some people don't.",/r/type2diabetes/comments/lsrxyw/new_here/goxee5q/,NaN,Topic 12,2021-02
85261,goxfko5,AccidentalDragon,2021-02-26 18:36,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,/r/type2diabetes/comments/lsrxyw/new_here/goxfko5/,NaN,Topic 5,2021-02
85262,lthxwe,Wackydetective,2021-02-26 22:17,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,/r/type2diabetes/comments/lthxwe/54_a1c/,5.4 A1C!!!,Topic 8,2021-02
85263,lu13pm,Nervous_Finger1486,2021-02-27 16:27,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,/r/type2diabetes/comments/lu13pm/still_new_to_this/,Still new to this.....,Topic 8,2021-02


In [15]:
#convert object to string
diabetes_all['Text'] = diabetes_all['Text'].astype(str)

In [17]:
#sentiment analysis with textblob
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d

#load the descriptions into textblob
desc_blob = [TextBlob(desc) for desc in diabetes_all['Text']]

#add the sentiment metrics to the dataframe
diabetes_all['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
diabetes_all['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

diabetes_all.head()

,Post ID,Author,Publish Date,Text,Permalink,Title,Dominant,month_year,tb_Pol,tb_Subj
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,Topic 4,2018-01,0.112973,0.552652
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",Topic 4,2018-01,0.005572,0.433556
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,Topic 4,2018-01,0.201414,0.588510
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,Topic 1,2018-03,0.095000,0.437778
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,Topic 1,2018-05,-0.081250,0.702500


In [18]:
#load VADER
analyzer = SentimentIntensityAnalyzer()

#Add VADER metrics to dataframe
diabetes_all['compound'] = [analyzer.polarity_scores(v)['compound'] for v in diabetes_all['Text']]
diabetes_all['neg'] = [analyzer.polarity_scores(v)['neg'] for v in diabetes_all['Text']]
diabetes_all['neu'] = [analyzer.polarity_scores(v)['neu'] for v in diabetes_all['Text']]
diabetes_all['pos'] = [analyzer.polarity_scores(v)['pos'] for v in diabetes_all['Text']]

diabetes_all.tail(50)

,Post ID,Author,Publish Date,Text,Permalink,Title,Dominant,month_year,tb_Pol,tb_Subj,compound,neg,neu,pos
85215,goqws6m,BafangFan,2021-02-25 12:13,Carnivore diet is probably the easiest diet when you're limited on cooking and storage. You can ...,/r/type2diabetes/comments/lscsfz/type_2_diet_with_very_little/goqws6m/,NaN,Topic 14,2021-02,0.276190,0.580952,0.6808,0.038,0.833,0.129
85216,goqzwdr,modernjess,2021-02-25 12:34,"For pizza, I've found most low-carb pizza crusts to be awful. The best ones I've found have been...",/r/type2diabetes/comments/lscsfz/type_2_diet_with_very_little/goqzwdr/,NaN,Topic 14,2021-02,0.166667,0.600000,0.2960,0.124,0.702,0.174
85217,gorg5o5,alan_s,2021-02-25 14:28,"Apply this technique to the foods available to you now (click on it): [Test, Review, Adjust](",/r/type2diabetes/comments/lscsfz/type_2_diet_with_very_little/gorg5o5/,NaN,Topic 6,2021-02,-0.175000,0.700000,0.0000,0.000,1.000,0.000
85218,lsfcp6,RiChessReadit,2021-02-25 11:58,"So I've been eating pretty much the same lunch for the last few weeks, a low carb wheat bread ha...",/r/type2diabetes/comments/lsfcp6/extremely_tired_and_feeling_hypo_normal_bg/,Extremely tired and feeling hypo- normal BG,Topic 5,2021-02,0.030000,0.365283,-0.5956,0.082,0.878,0.040
85219,gor0kig,modernjess,2021-02-25 12:39,"IME, it's not uncommon to get occasional errant readings. Maybe do a second reading? In any case...",/r/type2diabetes/comments/lsfcp6/extremely_tired_and_feeling_hypo_normal_bg/gor0kig/,NaN,Topic 13,2021-02,0.170000,0.625000,0.7845,0.000,0.769,0.231
85220,gor0ydz,RiChessReadit,2021-02-25 12:41,"I checked twice with different fingers, first reading was 110 the other was 109. The higher read...",/r/type2diabetes/comments/lsfcp6/extremely_tired_and_feeling_hypo_normal_bg/gor0ydz/,NaN,Topic 2,2021-02,0.184375,0.519792,0.2516,0.067,0.817,0.116
85221,gorgvns,alan_s,2021-02-25 14:33,"&gt; whole lunch is around 40-50g net carbs. \n\nTomorrow check every 30 minutes (30, 60, 90) af...",/r/type2diabetes/comments/lsfcp6/extremely_tired_and_feeling_hypo_normal_bg/gorgvns/,NaN,Topic 5,2021-02,-0.078000,0.401333,0.0000,0.000,1.000,0.000
85222,lsrhm7,MollyMooSkadoo,2021-02-25 22:12,I just need to rant. This might not be coherent.\n\nI was diagnosed this week. I was put on metf...,/r/type2diabetes/comments/lsrhm7/a_rant/,A Rant,Topic 8,2021-02,0.018177,0.528664,-0.9924,0.152,0.767,0.081
85223,gosxf7x,BeerLaoDrinker,2021-02-25 22:29,&amp;#x200B;\n\n&gt;Why not just take the medicine and hope it works well enough for a while?\n\...,/r/type2diabetes/comments/lsrhm7/a_rant/gosxf7x/,NaN,Topic 12,2021-02,0.124150,0.683673,0.8738,0.033,0.852,0.115
85224,gosxsmn,MollyMooSkadoo,2021-02-25 22:34,"I wish. I'm unfortunately infertile. I have diabetes because I have PCOS, and didn't know that p...",/r/type2diabetes/comments/lsrhm7/a_rant/gosxsmn/,NaN,Topic 6,2021-02,-0.400000,0.700000,-0.7003,0.188,0.752,0.060


In [19]:
#drop textblob columns
diabetes_all = diabetes_all.drop(columns = ['tb_Pol', 'tb_Subj'])
diabetes_all

,Post ID,Author,Publish Date,Text,Permalink,Title,Dominant,month_year,compound,neg,neu,pos
0,7ng7rl,penguinsarefun,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,/r/diabetes/comments/7ng7rl/t2_newbie/,T2 Newbie,Topic 4,2018-01,0.5319,0.087,0.797,0.116
1,7nlbda,MrCanoe,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,"Looking into Keto diet, how serious are risks of hypoglycemia?",Topic 4,2018-01,-0.9600,0.108,0.846,0.046
2,7nnmor,Heylola2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Great start to the new year!!,Topic 4,2018-01,0.9133,0.071,0.787,0.142
3,dwcllds,[deleted],2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,Topic 1,2018-03,0.8477,0.048,0.764,0.189
4,dzokqh6,Heylola2,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,Topic 1,2018-05,0.0000,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
85260,goxee5q,satxlonghorn1,2021-02-26 18:26,"I didn't have any side effects, just fyi. Some people don't.",/r/type2diabetes/comments/lsrxyw/new_here/goxee5q/,NaN,Topic 12,2021-02,0.2023,0.000,0.847,0.153
85261,goxfko5,AccidentalDragon,2021-02-26 18:36,Fasting brought my numbers was down from where they were! I do keto omad and came down from 160-...,/r/type2diabetes/comments/lsrxyw/new_here/goxfko5/,NaN,Topic 5,2021-02,0.0000,0.000,1.000,0.000
85262,lthxwe,Wackydetective,2021-02-26 22:17,I've been battling depression and anxiety like so many of us are living in end times. I was so a...,/r/type2diabetes/comments/lthxwe/54_a1c/,5.4 A1C!!!,Topic 8,2021-02,0.9092,0.083,0.724,0.192
85263,lu13pm,Nervous_Finger1486,2021-02-27 16:27,So it’s been a week since I was diagnosed. Metformin 1000mg twice a day. BS has been below 200 l...,/r/type2diabetes/comments/lu13pm/still_new_to_this/,Still new to this.....,Topic 8,2021-02,-0.7163,0.120,0.805,0.075


In [20]:
#download data with topics and sentiment score
diabetes_all.to_csv('diabetes_all_sentiment.csv', index = False)

In [21]:
#group average sentiment by topic
average_sentiment = diabetes_all.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment

<ipython-input-21-9e2281b3f4f7>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment = diabetes_all.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
Dominant,,,,
Topic 1,0.331763,0.053552,0.801744,0.144700
Topic 10,0.283063,0.060037,0.802462,0.137501
Topic 11,0.029692,0.091596,0.808070,0.100334
Topic 12,0.143915,0.062491,0.847392,0.090118
Topic 13,0.228108,0.060879,0.837639,0.101487
Topic 14,0.334538,0.051307,0.827873,0.120820
Topic 2,0.277266,0.044324,0.846078,0.109594
Topic 3,0.134357,0.059789,0.853994,0.086215
Topic 4,0.165952,0.071205,0.826378,0.102415


In [22]:
average_sentiment.to_csv('diabetes_all_sentiment_by_topic.csv', index = False)

In [23]:
#group by month 
average_sentiment_month = diabetes_all.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment_month

<ipython-input-23-80a851755b42>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment_month = diabetes_all.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
month_year,,,,
2018-01,0.223043,0.064977,0.815178,0.119868
2018-02,0.241664,0.058345,0.827951,0.113692
2018-03,0.238572,0.060657,0.826178,0.113168
2018-04,0.272012,0.058946,0.822487,0.118561
2018-05,0.224192,0.056669,0.836342,0.106993
2018-06,0.233440,0.055937,0.829753,0.114322
2018-07,0.242391,0.056879,0.827922,0.115193
2018-08,0.244742,0.058562,0.815929,0.125502
2018-09,0.244552,0.062213,0.817496,0.120272


In [24]:
average_sentiment_month.to_csv('diabetes_all_sentiment_by_month.csv', index = False)

In [25]:
#sort by authors & dates
diabetes_authors = diabetes_all.sort_values(by=['Author', 'Publish Date'])
diabetes_authors

,Post ID,Author,Publish Date,Text,Permalink,Title,Dominant,month_year,compound,neg,neu,pos
413,duo9a0o,#NAME?,2018-02-22 13:40,You need to do blood tests to determine which type of diabetes you have. They will look for the ...,/r/diabetes/comments/7zhpko/released_from_icu_1_week_ago_after_dka_newly/duo9a0o/,NaN,Topic 12,2018-02,-0.8516,0.160,0.786,0.054
2852,e1wik7j,#NAME?,2018-07-06 10:37,Congrats! Now time to lower the range.,/r/diabetes/comments/8wfk4i/three_days_in_the_green/e1wik7j/,NaN,Topic 3,2018-07,0.3595,0.202,0.459,0.339
4396,ejs9c61,#NAME?,2019-03-30 21:05,"I've been on it only for two weeks, but have had no adverse effects so far. My blood sugar has b...",/r/diabetes/comments/akajpr/afraid_to_start_trulicity/ejs9c61/,NaN,Topic 5,2019-03,0.7694,0.065,0.804,0.131
6396,emx2713,#NAME?,2019-05-09 6:21,I'm allergic to skin-tac so I just use medical tape. Got like 200 meters of it on amazon for $10,/r/diabetes/comments/bmjk9l/a_solution_for_cgm_sensor_adhesive_coming_unstuck/emx2713/,NaN,Topic 2,2019-05,0.2500,0.094,0.726,0.179
13714,fop3oat,#NAME?,2020-04-26 18:52,I felt that a little too hard...\n\nI hate it too.,/r/diabetes/comments/g8ljrg/sometimes_when_i_think_i_am_having_an_anxiety/fop3oat/,NaN,Topic 8,2020-04,-0.5888,0.349,0.651,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
61544,g4e4407,zymurgy_bc,2020-09-07 16:55,"Well, I think it's definitely ""your mileage may vary"" and no one really says otherwise. Glycemi...",/r/diabetes_t2/comments/ins6jh/understanding_bs_spikes_3_meals_showns_guess_all/g4e4407/,NaN,Topic 13,2020-09,0.9639,0.101,0.741,0.158
61547,g4ebzvy,zymurgy_bc,2020-09-07 18:09,I think he's less than 6 months in... no idea how much education has been taken in yet. My doct...,/r/diabetes_t2/comments/ins6jh/understanding_bs_spikes_3_meals_showns_guess_all/g4ebzvy/,NaN,Topic 10,2020-09,-0.6640,0.075,0.884,0.041
61537,g4bv31y,zymurgy_bc,2020-09-07 6:39,Agreed on this! Some bakeries will replace the gluten with other things since their primary goal...,/r/diabetes_t2/comments/ins6jh/understanding_bs_spikes_3_meals_showns_guess_all/g4bv31y/,NaN,Topic 14,2020-09,0.8217,0.000,0.765,0.235
62078,g54ypi0,zymurgy_bc,2020-09-13 10:50,"Not according to Trump... and he's a stable genius, unlike them fact checking fake news sources ...",/r/diabetes_t2/comments/iry0zu/i_had_an_addiction_to_cheese_that_was_harder_to/g54ypi0/,NaN,Topic 10,2020-09,0.4294,0.119,0.653,0.228


In [26]:
diabetes_authors.to_csv('diabetes_all_authors_sentiment.csv', index = False)